In [ ]:
!pip install lime

In [ ]:
import tensorflow as tf
import numpy as np
import PIL.Image
from skimage.io import imread
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
from lime import lime_image

%matplotlib inline

In [ ]:
def LoadImage(file_path, model='VGG'):
    im = PIL.Image.open(file_path)
    if model == 'Inception':
        im = im.resize((299,299))
    else:
        im = im.resize((224,224))
    im = np.asarray(im)
    return im

def PreprocessVGGImage(im):
    im = tf.keras.applications.vgg16.preprocess_input(im)
    return im

def PreprocessInceptionImage(im):
    im = tf.keras.applications.inception_v3.preprocess_input(im)
    return im


In [ ]:
im_orig = LoadImage('download.jpg', model='Inception')

plt.imshow(im_orig)
plt.show()

In [ ]:
from skimage.segmentation import felzenszwalb, quickshift

segments = quickshift(im_orig,
                      kernel_size=4,
                      max_dist=200,
                      ratio=0.2,
                      rng=42)
plt.imshow(segments)
plt.show()

In [ ]:
plt.imshow(mark_boundaries(im_orig, segments))
plt.show()

In [ ]:
segmented_image = im_orig.copy()

# Isolate a few segments
segmented_image[segments == 1] = 0
segmented_image[segments == 2] = 0
segmented_image[segments == 3] = 0
segmented_image[segments == 4] = 0
segmented_image[segments == 5] = 0
segmented_image[segments == 6] = 0
plt.imshow(segmented_image)
plt.show()

In [ ]:
fudged_image = im_orig.copy()
# Isolate a few segments
fudged_image[segments == 1] = 0
fudged_image[segments == 2] = 0
fudged_image[segments == 3] = 0
fudged_image[segments == 4] = 0
fudged_image[segments == 5] = 100
plt.imshow(fudged_image)
plt.show()

In [ ]:
segmented_image = im_orig.copy()
# Replace superpixel with average of the pixel values
for x in np.unique(segments):
    segmented_image[segments == x] = (
        np.mean(im_orig[segments == x][:, 0]),
        np.mean(im_orig[segments == x][:, 1]),
        np.mean(im_orig[segments == x][:, 2]))

plt.imshow(segmented_image)
plt.show()

In [ ]:
inception = tf.keras.applications.InceptionV3(
    include_top=True, weights='imagenet')
model = tf.keras.models.Model(inception.inputs, inception.output)

In [ ]:
# Load the image
im_orig = LoadImage('download.jpg', model='Inception')
image = PreprocessInceptionImage(im_orig)

# Show the image
plt.imshow(im_orig)

predictions = model(np.array([image]))
prediction_class = np.argmax(predictions[0])

print("Prediction class: " + str(prediction_class))

In [ ]:
# Create LIME explainer
explainer = lime_image.LimeImageExplainer()

# Define prediction function for LIME
def predict_fn(images):
    preprocessed = np.array([PreprocessInceptionImage(img) for img in images])
    return model.predict(preprocessed)

# Generate explanation
explanation = explainer.explain_instance(
    im_orig,
    predict_fn,
    top_labels=5,
    hide_color=0,
    num_samples=1000
)

print("Explanation generated successfully!")
print("Top labels:", explanation.top_labels)

In [ ]:
temp, mask = explanation.get_image_and_mask(label=explanation.top_labels[0],
                                            positive_only=False,
                                            num_features=10,
                                            hide_rest=True)
plt.imshow(mark_boundaries(temp, mask))
plt.show()